#Imports


In [0]:
import pandas as pd
from unidecode import unidecode
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr, mannwhitneyu, spearmanr
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Carregando dados e criando Metadados

In [0]:
file_path = 'data/Seguro Saúde - Modelagem.xlsx'

df = pd.read_excel(file_path, sheet_name='MODELAGEM')
dicionario = pd.read_excel(file_path, sheet_name='DICIONÁRIO')

# Função que deixa colunas minúsculas
def normalize_columns(df):
    df.columns = [unidecode(col).lower() for col in df.columns]
    return df

# Função para normalizar os valores das colunas de texto
def normalize_text_values(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: unidecode(x).lower() if isinstance(x, str) else x)
    return df

df = normalize_columns(df)
df = normalize_text_values(df)

dicionario = normalize_columns(dicionario)
dicionario = normalize_text_values(dicionario)

dicionario.columns = ['variavel', 'descricao']

display(df.head())
display(dicionario.head(10))

In [0]:
df.filhos = df.filhos.astype('Int64')
df.fumante = df.fumante.astype('Int64')
df.regiao = df.regiao.astype('Int64')
df.facebook = df.facebook.astype('Int64')
df.classe = df.classe.astype('Int64')
display(df.head())

In [0]:
def percent_nulls(df):
    return round(df.isnull().mean() * 100, 2)

def unique_values(df):
    return df.nunique()

def percent_zeros(df):
    return round((df == 0).mean() * 100, 2)

# Criando meta dados de df
metadata = pd.DataFrame({
    'variavel': df.columns,
    'tipo': df.dtypes.astype(str),
    'percentual_nulos': percent_nulls(df),
    'valores_unicos': unique_values(df),
    'percentual_zeros': percent_zeros(df)
})

metadata.reset_index(drop=True, inplace=True)

display(metadata)
metadata.to_csv('artifacts/metadata.csv', index=False)

# Funçoes

In [0]:

def plot_categorical(df, var, segmento=''):
    # Criar uma cópia da coluna para evitar modificar o DataFrame original
    data = df[var].copy()
    
    # Substituir 0 por 'não' e 1 por 'sim' para variáveis binárias
    if set(data.dropna().unique()) == {0, 1}:
        data = data.map({0: 'não', 1: 'sim'})
    
    # Calcular os percentuais
    value_counts = data.value_counts(normalize=True) * 100
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=value_counts.index, y=value_counts.values, palette='viridis', order=value_counts.index)
    plt.title(f'{var}' + (f' - {segmento}' if segmento else ''))
    plt.xlabel(var)
    plt.ylabel('Percentual (%)')
    plt.xticks(rotation=45)
    
    # Incluir o valor do percentual dentro da barra
    for i, value in enumerate(value_counts.values):
        plt.text(i, value / 2, f'{value:.2f}%', ha='center', va='center', color='white', fontweight='bold')
    
    plt.tight_layout()
    
    # Salvar o gráfico
    plt.savefig(f'plots/percentual_{var}' + (f'_{segmento}' if segmento else '')  + '.png')
    
    # Exibir o gráfico
    plt.show()

In [0]:
def plot_numeric(df, var, segmento=''):
    # Remover valores NaN e infinitos
    data = df[var].replace([np.inf, -np.inf], np.nan).dropna().to_numpy()
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Histograma
    sns.histplot(data, kde=True, ax=axes[0], color='skyblue')
    axes[0].set_title(f'Histograma de {var}' + (f' - {segmento}' if segmento else ''))
    axes[0].set_xlabel(var)
    axes[0].set_ylabel('Frequência')
    
    # Boxplot
    sns.boxplot(y=data, ax=axes[1], color='lightgreen')
    axes[1].set_title(f'Boxplot de {var}' + (f' - {segmento}' if segmento else ''))
    axes[1].set_xlabel(var) 
    axes[1].set_ylabel(var)
    
    plt.tight_layout()
    
    path_to_save =( 
                   f'plots/histograma_boxplot_{var}' + (f'_{segmento}' if segmento else '') + '.png')
    plt.savefig(path_to_save)
    
    # Exibir o gráfico
    plt.show()

In [0]:
def plot_scatter(df, var, faixa=''):
    df = df.dropna()
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=var, y='valor', data=df)

    # Calcular a linha de tendência (regressão linear)
    coeffs = np.polyfit(df[var], df['valor'], 1)
    line = np.poly1d(coeffs)

    # Plotar a linha de tendência
    plt.plot(df[var], line(df[var]), color='red', linestyle='--', 
            label=f'Linha de Tendência: y = {coeffs[0]:.2f}x + {coeffs[1]:.2f}')

    plt.title(f'{var} vs Valor' + (f' - Faixa de valor: {faixa}' if faixa else ''))
    plt.xlabel(var)
    plt.ylabel('Valor')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'plots/scatter_{var}_valor' + (f'_{faixa}' if faixa else '') + '.png')
    plt.show()

# Análise Univariada

Os custos anuais por cliente:
- Custos chegam até R$19.500,00, mas sao casos esporádicos
  - skewness a direita; cauda longa
- Média de R$ 4.000,00 de gasto anual por cliente
- maior concentraçao em valores até R$ 4.500,00
- Apenas 25% dos clientes supepram R$ 4.800,00 de custo anual

In [0]:
display(df.valor.describe())

# Criar o histograma
plt.figure(figsize=(10, 6))
sns.histplot(df.valor, kde=True)
plt.title('Distribuição dos Gastos Anuais')
plt.xlabel('Valor (R$)')
plt.ylabel('Frequência')

# Salvar o gráfico
plt.savefig('plots/histograma_valor.png')

# Exibir o gráfico
plt.show()

In [0]:
data = df['valor'].replace([np.inf, -np.inf], np.nan).dropna()

# Quartis
Q1 = data.quantile(0.25)
Q2 = data.quantile(0.50)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

print(f'Q1 da variável valor: {Q1}')
print(f'Q2 da variável valor: {Q2}')
print(f'Q3 da variável valor: {Q3}')
print(f'IQR da variável valor: {IQR}')

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(y=data, color='lightgreen')
plt.title('Boxplot de valor')
plt.ylabel('Valor (R$)')

plt.axhline(Q1, color='r', linestyle='--', label='Q1')
plt.axhline(Q2, color='g', linestyle='-', label='Q2 (Mediana)')
plt.axhline(Q3, color='b', linestyle='--', label='Q3')
plt.legend()

plt.tight_layout()

plt.savefig('plots/boxplot_valor.png')

plt.show()


Pelo boxplot, vemos:
- um IQR de aproximadamente R$ 34.400,00
- Podemos classificar os clcientes eme 4 categogrias
  - muito baratos: custos até R$ 1.435,00
  - baratos: R$ 1.435,00 < custos > R$ 2.800,00
  - Caros: R$ 2.800,00 < custos > RS 4.805,00
  - Muito caros: custos > R$ 4.805,00

**OBS:** Esta classifiçao nao tem o objetivo de tratar o problema como classificaçao

Agora uma análise de correlaçao das numéricas com a variável resposta

In [0]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.drop('matricula', axis=1).corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

plt.savefig('plots/matriz_correlacao.png')

Vamo que apenas a variável fumante tem alta correlaçao com o valor anual e que há baixíssimo correlaçao entre as variáveis

## Categóricas
Queremos agora observar as distribuiçoes das  variáveis categóricas que sao visualmente simples de se interpretar

In [0]:
df_no_matricula = df.drop(columns=['matricula'])

# somente categóricas
categorical_vars = df_no_matricula.select_dtypes(include=['object', 'int64']).columns

for var in categorical_vars:
    plot_categorical(df_no_matricula, var)

Agora faremos uma análise do valor médio anual por categoria das categóricas

In [0]:
df.groupby('regiao')['valor'].mean().sort_values().plot(kind='barh')
plt.title('Média de Valor por Região')
plt.xlabel('Valor Médio')
plt.show()

plt.savefig('plots/media_valor_regiao.png')

In [0]:
df.groupby('classe')['valor'].mean().sort_values().plot(kind='barh')
plt.title('Média de Valor por Classe')
plt.xlabel('Valor Médio')
plt.show()

plt.savefig('plots/media_valor_classe.png')

In [0]:
df.groupby('sexo')['valor'].mean().sort_values().plot(kind='barh')
plt.title('Média de Valor por Sexo')
plt.xlabel('Valor Médio')
plt.show()

plt.savefig('plots/media_valor_sexo.png')

In [0]:
df.groupby('fumante')['valor'].mean().sort_values().plot(kind='barh')
plt.title('Média de Valor por Status de Fumante')
plt.xlabel('Valor Médio')
plt.show()

plt.savefig('plots/media_valor_fumante.png')

In [0]:
df.groupby('facebook')['valor'].mean().sort_values().plot(kind='barh')
plt.title('Média de Valor por Status de usuário de Facebook')
plt.xlabel('Valor Médio')
plt.show()

plt.savefig('plots/media_valor_facebook.png')

In [0]:
df.groupby('signo')['valor'].mean().sort_values().plot(kind='barh')
plt.title('Média de Valor por Signo')
plt.xlabel('Valor Médio')
plt.show()

plt.savefig('plots/media_valor_signo.png')

## Numéricas
Queremos avaliar agora as variáveis numéricas, o que inclui:
- histogramas e boxplots
- análises segmentadas por variáveis categóricas como sexo, fumante, classe

In [0]:
plot_numeric(df, 'idade')
plot_numeric(df, 'imc')


Analisando os gráficos acima e os abaixo segmentados por sexo, vemos que a distribuiçao de idade e IMC nao diferem muito entre homens e mulheres

In [0]:
plot_numeric(df[df.sexo== 'm'], 'idade', 'masculino')
plot_numeric(df[df.sexo== 'f'], 'idade', 'feminino')
plot_numeric(df[df.sexo== 'm'], 'imc', 'masculino')
plot_numeric(df[df.sexo== 'f'], 'imc', 'feminino')


Vejamos agora o IMC geral segundo a classificaçao do ministérioo da saúde

In [0]:
df['imc_categoria'] = pd.cut(
    df['imc'],
    bins=[-float('inf'), 16, 16.9, 18.4, 24.9, 29.9, 34.9, 39.9, float('inf')],
    labels=['magreza grave', 'magreza moderada', 'magreza leve', 'normal', 'sobrepeso', 'obesidade grau 1', 'obesidade severa', 'obesidade mórbida']
)

plot_categorical(df[df.fumante == 1], 'imc_categoria', 'fumante')
plot_categorical(df[df.fumante == 0], 'imc_categoria', 'não fumante')


Observamos que nao há diferença significativa na distribuiçao do grau de IMC e ser fumante ou nao


#Análise Multiivariada


Queremos observar se há relaçao entre a idade e os custos anuais

In [0]:
plot_scatter(df, 'idade')


Observamos 3 grupos de relaçao por faixa de valores:
- até R$ 5.000,00
- entre  R$ 5.000,00 e R$ 10.000,00
- gastos maiores que R$ 10.000,00

Vamos explorar graficamente os 3 grupos e analisar a tendência de correlaçao

In [0]:
aux = df.dropna()
aux1 = aux[aux.valor <= 5000]
aux2 = aux[(aux.valor > 5000) & (aux.valor <= 10000)]
aux3 = aux[aux.valor > 10000]

plot_scatter(aux1, 'idade', '<=5000')
r, p_value = pearsonr(aux1['idade'], aux1['valor'])
print(f"Cluster {'<= R$ 5.000,00'}: Correlação (r) = {r:.3f}, p-value = {p_value:.3f}")

plot_scatter(aux2, 'idade', '5000-10000')
r, p_value = pearsonr(aux2['idade'], aux2['valor'])
print(f"Cluster {'> R$ 5.000,00 <= R$ 10.000,00'}: Correlação (r) = {r:.3f}, p-value = {p_value:.3f}")

plot_scatter(aux3, 'idade', '>10000')
r, p_value = pearsonr(aux3['idade'], aux3['valor'])
print(f"Cluster {'> R$ 10.000,00'}: Correlação (r) = {r:.3f}, p-value = {p_value:.3f}")


Concluimos que para as 3 faixas de valores temos altas correlaçoes e estatisticamente sigificativas como indicam os p-values, destacando a alta correlaçao entre a idade e os gastos para clientes com gastos de barato a moderado - até R$5.000,00

Segue os coeficientes de correlaçao:
- Até R$ 5.000,00: 0.85 (altíssima correlaçao)
- Entre R$ 5.000,00 e R$ 10.000,00: 0.65 (correlaçao consideravel)
- Acima  de R$ 10.000,00: 0.61 (também considerável)


Vejamos agora a relaçao entre imc e os gastos

In [0]:
aux = df.replace([np.inf, -np.inf], np.nan).dropna()
plot_scatter(aux, 'imc')
r, p_value = pearsonr(aux['imc'], aux['valor'])
print(f"IMC vs Valor: Correlação (r) = {r:.2f}, p-value = {p_value:.2f}")


Mais uma vez parece que a relaçao se estabelece por faixa de valor. Verifiquemos, portanto.

In [0]:
aux = df.dropna()
aux1 = aux[aux.valor <= 5000]
aux2 = aux[aux.valor > 5000]

plot_scatter(aux1, 'imc', '<=5000')
r, p_value = pearsonr(aux1['imc'], aux1['valor'])
print(f"Cluster {'<= R$ 5.000,00'}: Correlação (r) = {r:.3f}, p-value = {p_value:.3f}")

plot_scatter(aux2, 'imc', '>5000')
r, p_value = pearsonr(aux2['imc'], aux2['valor'])
print(f"Cluster {'> R$ 5.000,00'}: Correlação (r) = {r:.3f}, p-value = {p_value:.3f}")


Vemos que para valores maiores que R$ 5.000,00, o IMC tem uma correlaçao considerável com os custos anuais. Já para gastos de baixo a moderado, nao há correlaçao entre IMC e gasto anual.


Queremos agora analisar a relaçao dos gastos com o fato de ser fumante. Para isso comparemos os boxplot de gastos para cada status de fumante

In [0]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=df.dropna(), x='fumante', y='valor', palette='viridis')
plt.title('Distribuição do Custo por Status de Fumante', fontsize=14)
plt.xlabel('Fumante (0 = Não, 1 = Sim)', fontsize=12)
plt.ylabel('Custo (R$)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()


Vemos uma diferrença muito grande entre as distribuiçoes de gastos com os fumantes acaretando uma mediana de gastos acima dos R$ 10.000,00 enquanto que para nao fumantes gastos próximos a R$ 10,000 sao outliers. Portanto, esta valor preditivo forte.

In [0]:
plt.figure(figsize=(10, 6))

sns.boxplot(data=df.dropna(), x='regiao', y='valor', palette='coolwarm')

plt.title('Distribuição do Custo por Região', fontsize=14)
plt.xlabel('Região', fontsize=12)
plt.ylabel('Custo (R$)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()


Vemos que nao há diferenças entre os gastos e as regioes, destacando-se apenas a regiao 1 com um IQR pouco mais largo e valor máximo maior. Mas nao chega a apresentar uma distribuiçao que se diferencice bastante das outras regioes. 


Agora vejamos a distribuiçao da variável classe vs valor. Usaremos violinplot destacando os quartis de classe, já que classe tem alta cardinalidade e, portanto, requer uma visuaalizaçao menos "poluída" e mais condensada

In [0]:
plt.figure(figsize=(10, 6))

sns.violinplot(data=df.dropna(), x='classe', y='valor', inner='quartile')  # Mostra quartis dentro do violino

plt.title('Distribuição do Custo por Classe de Clientes', fontsize=14)
plt.xlabel('Classe', fontsize=12)
plt.ylabel('Custo (R$)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()



Vemos que nao há diferenças cocnsideráveis na relaçao valor-classe

In [0]:
plt.figure(figsize=(10, 6))
aux = df.dropna()
aux['facebook'] = aux.facebook.map({0: 'não', 1: 'sim'})
sns.boxplot(data=aux, x='facebook', y='valor', palette='coolwarm')

plt.title('Distribuição do Custo por Status de Usuário do Facebook', fontsize=14)
plt.xlabel('Usuário Facebook', fontsize=12)
plt.ylabel('Custo (R$)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()


Parece haver uma diferença entre  usuários e nao usuários  do facebook pelas medianas  pouco diferentes,  iqrs com larguras ligieramente distintas e distribuiçaoes com quartis diferentes. Façamos um teste de hipótese  pra checar se  a diferença é estatisticamemnte significativa ou nao entre os dois  casos

In [0]:
grupo_sim = df.dropna()[df.facebook == 1]['valor']
grupo_nao = df.dropna()[df.facebook == 0]['valor']

stat, p_value = mannwhitneyu(grupo_nao, grupo_sim, alternative='two-sided')
print(f"Mann-Whitney: p-value = {p_value:.4f}")


Há uma diferença significativa entre as distribuiçoes de gastos entre usuáŕios e nao usuárioss do facebook, indicando que usáŕios têm umam distribuicao de gastos menor.

Isso pode terr a ver com  a idade  dos clientes, uma vez que usuários mais jovens tendem a usar redes sociais mais que os mais idosos. E vimos antes que para 3  faixas de valores anuais diferentes quanto mais velho o cliente maior seus gastos.


Vejamos agora a variável filhos e distribuiçao de valor anual por quanntidade de filhos do cliente. 
Analisemos:
- Violin plots, por ter 6 valores
- Spearman correlation pra checar a correlaçao ordinal entre as duas variáveis

In [0]:
corr, p_value = spearmanr(df.dropna()['filhos'], df.dropna()['valor'])
print(f"Correlação de Spearman: r = {corr:.3f}, p-value = {p_value:.4f}")

plt.figure(figsize=(10, 6))

sns.violinplot(data=df.dropna(), x='filhos', y='valor', inner='quartile')  # Mostra quartis dentro do violino

plt.title('Distribuição do Custo por Quantidade de filhos', fontsize=14)
plt.xlabel('Quantidade de filhos', fontsize=12)
plt.ylabel('Custo (R$)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()



Sem relaçao direta entre a idade do cliente o seu custo anual


Vejamos agora a relaçao entre os custos anuais por cliente e o signo. Apesar de, a princípio parecer uma variável aleatória com relaçao ao custo, ela indica, se é verdade que os signos apontam comportamento, que alguns clientes podem ter mais ou menos preocupaçoes com a saúde e trazerem mais ou menos gastos.

Vejamos os boxplots pra cada signo

In [0]:
plt.figure(figsize=(10, 6))

sns.violinplot(data=df.dropna(), x='signo', y='valor', inner='quartile')  # Mostra quartis dentro do violino

plt.title('Distribuição do Custo por Signo', fontsize=14)
plt.xlabel('Signos', fontsize=12)
plt.ylabel('Custo (R$)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()



Observamos que as distribuiçoes de custo por signo nao indicam padroes diferentes.

# Análise com algoritmo A Priori

## Categorizando os dados

In [0]:
df['faixa_etaria'] = pd.cut(df['idade'],
                           bins=[0, 30, 50, 100],
                           labels=['jovem', 'adulto', 'senior'])
df['categoria_qtd_filhos'] = pd.cut(df['filhos'],
                                   bins=[0,1,2,4,6],
                                   labels=['nenhum', 'poucos', 'medio', 'muitos'])
df['categoria_valor'] = pd.cut(df['valor'],
                              bins=[0, 2500, 5000, 10000, float('inf')],
                              labels=['barato', 'moderado', 'caro', 'muito_caro'])

cols_categoricas_inteiro = ['fumante', 'facebook', 'regiao', 'classe'] 

# Convertendo para tipo categórico
for col in cols_categoricas_inteiro:
    df[col] = df[col].astype('category')

In [0]:
display(df.head())

## Preparando os dados pro algoritmo

In [0]:
cols_analise = ['sexo', 'faixa_etaria', 'fumante', 'regiao',
                'facebook', 'classe', 'imc_categoria', 
                'categoria_qtd_filhos','categoria_valor']

df_analise = df[cols_analise].dropna()
df_analise = df_analise.astype(str)

# Transformando em transações
transacoes = df_analise.apply(lambda x: [f"{col}={val}" for col, val in zip(x.index, x.values)], axis=1).tolist()

# One-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transacoes).transform(transacoes)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

## Grid search do Algoritmo

In [0]:
# Experimentando com diferentes parâmetros
param_grid = {
    'min_support': [0.05, 0.1, 0.15],
    'min_threshold': [0.6, 0.7, 0.8]
}

valor_categories = ['categoria_valor=barato', 'categoria_valor=moderado', 
                   'categoria_valor=caro', 'categoria_valor=muito_caro']

results = []

for support in param_grid['min_support']:
    # Gerar itemsets frequentes
    fi = apriori(df_encoded, min_support=support, use_colnames=True)
    
    for threshold in param_grid['min_threshold']:
        # Gerar regras
        r = association_rules(fi, metric="confidence", min_threshold=threshold)
        
        # Filtrar regras relevantes
        r = r[r['consequents'].apply(lambda x: any(item in valor_categories for item in x))]
        
        if not r.empty:
            results.append({
                'min_support': support,
                'min_threshold': threshold,
                'num_rules': len(r),
                'avg_lift': r['lift'].mean(),
                'max_lift': r['lift'].max()
            })

# Analisar resultados dos parâmetros
param_results = pd.DataFrame(results)
print("\nResultados dos diferentes parâmetros:")
display(param_results.sort_values('avg_lift', ascending=False))

## Procurando regras de Associaçao

In [0]:
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [0]:
# itemsets mais frequentes
print("Itemsets mais frequentes:")
print(frequent_itemsets.sort_values('support', ascending=False).head(10))

print("\nDistribuição por tamanho de itemset:")
print(frequent_itemsets['length'].value_counts())

# itemsets com 2+ itens para encontrar associações
itemsets_interessantes = frequent_itemsets[frequent_itemsets['length'] >= 2]

In [0]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

rules_valor = rules[rules['consequents'].apply(lambda x: any(item in valor_categories for item in x))]

In [0]:
def analisar_categoria(rules, categoria):
    cat_rules = rules[rules['consequents'].apply(lambda x: f'categoria_valor={categoria}' in x)]
    
    cat_rules = cat_rules.sort_values(['lift', 'confidence'], ascending=[False, False])
    
    print(f"\n{'='*60}")
    print(f"REGRAS MAIS FORTES PARA: {categoria.upper()}")
    print(f"{'='*60}")
    
    for idx, (_, rule) in enumerate(cat_rules.head(10).iterrows(), 1):
        # Processar antecedentes
        antecedentes = []
        for item in rule['antecedents']:
            if '=' in item:
                var, val = item.split('=', 1)
                antecedentes.append(f"{var}={val}")
            else:
                antecedentes.append(item)
        
        # Processar consequente
        consequente = list(rule['consequents'])[0].split('=')[1]
        
        print(f"\nRegra #{idx}:")
        print("SE", " E ".join(antecedentes))
        print("ENTÃO", consequente)
        print(f"Suporte: {rule['support']:.3f} | Confiança: {rule['confidence']:.3f} | Lift: {rule['lift']:.3f}")
        print("-"*50)

    return cat_rules

In [0]:
rules_barato = analisar_categoria(rules_valor, 'barato')
rules_moderado = analisar_categoria(rules_valor, 'moderado')
rules_caro = analisar_categoria(rules_valor, 'caro')
rules_muito_caro = analisar_categoria(rules_valor, 'muito_caro')

In [0]:
plt.style.use('seaborn')
sns.set_palette("husl")

# Gráfico de dispersão para as regras
plt.figure(figsize=(12, 8))
scatter = sns.scatterplot(
    data=rules_valor,
    x='support',
    y='confidence',
    hue='consequents',
    size='lift',
    sizes=(20, 200),
    alpha=0.7
)
plt.title('Relação entre Suporte e Confiança das Regras por Categoria de Valor\n(Tamanho pelo Lift)', fontsize=14)
plt.xlabel('Suporte', fontsize=12)
plt.ylabel('Confiança', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [0]:
top_rules = rules_valor.sort_values('lift', ascending=False).head(10)
top_rules = top_rules.drop_duplicates(subset=['antecedents', 'consequents'])

top_rules['regra_completa'] = top_rules.apply(
    lambda row: " → ".join([
        ', '.join(sorted(row['antecedents'])), 
        list(row['consequents'])[0]
    ]), 
    axis=1
)

top_rules = top_rules.drop_duplicates(subset=['regra_completa'])

top_rules = top_rules.sort_values('lift', ascending=False).head(8)

plt.figure(figsize=(14, 8))
ax = plt.subplot()

bars = ax.barh(
    y=top_rules['regra_completa'],
    width=top_rules['lift'],
    color='#3498db',
    height=0.7,
    edgecolor='none'
)

plt.suptitle('Principais Regras de Associaçao Cliente vs Valor',
             x=0.3, 
             y=0.98,
             fontsize=20,
             fontweight='bold',
             ha='center')

ax.set_xlabel('Multiplicador de Eficácia (Lift)', fontsize=14)
ax.set_yticks(range(len(top_rules)))
ax.set_yticklabels(top_rules['regra_completa'], fontsize=12)
ax.invert_yaxis()

for bar in bars:
    width = bar.get_width()
    ax.text(
        width + 0.2, 
        bar.get_y() + bar.get_height()/2, 
        f'{width:.2f}x', 
        va='center', 
        ha='left',
        fontsize=12,
        color='#2c3e50'
    )

plt.subplots_adjust(left=0.45, right=0.85)
plt.box(False)
plt.show()

# Feechando o dataframe

In [0]:
df.head()

In [0]:
metadata = pd.DataFrame({
    'variavel': df.columns,
    'tipo': df.dtypes.astype(str),
    'percentual_nulos': percent_nulls(df),
    'valores_unicos': unique_values(df),
    'percentual_zeros': percent_zeros(df)
})

metadata.reset_index(drop=True, inplace=True)

display(metadata)
# metadata.to_csv('artifacts/metadata.csv', index=False)